In [52]:
import os
import yaml
from Experiments.EXP1.trainer import trainer, model_callByName, loss_callByName
from data_utils import get_uci_data, common_processor_UCI, seed_all, california_housing_process, normalize, splitter
from Experiments.EXP1.TestPerform import testPerform_muSigma, testPerform_kernel
import torch
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor


base_seed = 1234
num_repeat = 5
big_df = {}

err_mu_dic = {}
err_std_dic = {}

dataname = "parkinsons"


df = pd.read_csv(os.getcwd() + "/Dataset/UCI_datasets/parkinsons.txt", sep=",")
df1 = df[df["age"] < 74]
df2 = df[df["age"] >= 74]

ds1 = df1.iloc[:,5:].to_numpy()
ds2 = df2.iloc[:,5:].to_numpy()


x, y = ds1[:,1:], ds1[:,0]

# test_X, test_Y = ds3[:,1:], ds3[:,0]
test_X, test_Y = ds2[:,1:], ds2[:,0]

x_normed, x_normalizer = normalize(x)

x = x_normed
test_X = x_normalizer.transform(test_X)


modelname = "RFKernel"

for ratio in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:


    N_train = int(len(x)*ratio)
    N_recal = int(len(x)*(1-ratio))

    tr_idx, recal_idx = splitter(N_train, N_recal)
    train_X, train_Y = x[tr_idx], y[tr_idx]

    recal_X, recal_Y = x[recal_idx], y[recal_idx]

    train_X, test_X, recal_X= torch.Tensor(train_X), torch.Tensor(test_X), torch.Tensor(recal_X)
    train_Y, test_Y, recal_Y= torch.Tensor(train_Y).to(torch.device("cuda")), torch.Tensor(test_Y).to(torch.device("cuda")), torch.Tensor(recal_Y).to(torch.device("cuda"))

    depth = 20

    rf_model = RandomForestRegressor(max_depth=depth, random_state=0)
    rf_model.fit(train_X.cpu().numpy(), train_Y.cpu().numpy())


    record = testPerform_kernel(
        test_X,
        test_Y,
        recal_X,
        recal_Y,
        model_name = "RFKernel",
        model = rf_model,
        val_criterias = [
            "MACE_Loss", "AGCE_Loss", "CheckScore"
        ],
        wid = 1
    )
    print(ratio)
    print(record)

0.8
{'MACE_Loss': 0.11065573245286942, 'AGCE_Loss': 0.14792080223560333, 'CheckScore': 3.5193464756011963}
0.7
{'MACE_Loss': 0.11274012923240662, 'AGCE_Loss': 0.1497029811143875, 'CheckScore': 3.4977457523345947}
0.6
{'MACE_Loss': 0.11456424742937088, 'AGCE_Loss': 0.1679207980632782, 'CheckScore': 3.4910762310028076}
0.5
{'MACE_Loss': 0.10860825330018997, 'AGCE_Loss': 0.15069307386875153, 'CheckScore': 3.442072629928589}
0.4
{'MACE_Loss': 0.11018094420433044, 'AGCE_Loss': 0.15683168172836304, 'CheckScore': 3.4513792991638184}
0.3
{'MACE_Loss': 0.11717885732650757, 'AGCE_Loss': 0.1528712958097458, 'CheckScore': 3.460111379623413}
0.2
{'MACE_Loss': 0.12181782722473145, 'AGCE_Loss': 0.15920792520046234, 'CheckScore': 3.4061572551727295}


array([34.398, 34.894, 35.389, ..., 32.495, 32.007, 31.513])